In [146]:
import json
import os
from collections import defaultdict
from pathlib import Path

from termcolor import cprint

from poe_types import PoEItem, PoEMod
from utils import loadRecombsFromFileList, validateAndReturn

In [131]:
# Load recomb files for easy PoEItems
json_dir = Path().parent / 'data/json'
recombination_files = sorted(os.listdir(json_dir))
recombination_files_full = [json_dir / name for name in recombination_files]
recombs = loadRecombsFromFileList(recombination_files_full)
print(len(recombs))

264


In [132]:
data_path = Path('../RePoE/RePoE/data').resolve()

In [133]:
with open(data_path / 'mods.json') as f:
    j = json.load(f)
jkl = list(j.keys())
print(len(jkl))
print(jkl[:50])

36720
['Strength1', 'Strength2', 'Strength3', 'Strength4', 'Strength5', 'Strength6', 'Strength7', 'Strength8', 'Strength9', 'Strength10', 'StrengthEssence7_', 'StrengthUniqueRing2', 'StrengthImplicitAmulet1', 'StrengthImplicitBelt1', 'StrengthImplicitBeltRoyale1', 'StrengthUniqueHelmetDexInt1', 'StrengthUniqueTwoHandMace1', 'StrengthUniqueAmulet5', 'StrengthUniqueIntHelmet3', 'StrengthUniqueBootsInt1', 'StrengthUniqueDagger2', 'StrengthUniqueBootsStrDex1', 'StrengthUniqueBootsStrDex1Royale', 'StrengthUniqueGlovesDex1', 'StrengthUniqueBelt1', 'StrengthUniqueBelt2', 'StrengthUniqueBelt4', 'StrengthUniqueGlovesStr2', 'StrengthUniqueTwoHandAxe3', 'StrengthUniqueBodyStrInt3', 'StrengthUniqueGlovesStrDex3', 'StrengthUniqueHelmetStrDex3', 'StrengthUniqueClaw5_', 'StrengthUniqueRing8', 'StrengthUniqueBootsDexInt2', 'StrengthUniqueTwoHandAxe5', 'StrengthUniqueTwoHandSword5', 'StrengthUniqueBelt7', 'StrengthUniqueSceptre6', 'StrengthUniqueBodyStr4', 'StrengthUniqueQuiver6', 'StrengthUniqueOneHan

In [134]:
# Mods from RePoE don't have int before (), but mods from in game do
# Need to handle both cases
mod_text_examples = [
    '(55-64)% increased Physical Damage\n+(124-149) to Accuracy Rating',
    'Adds 11(11-14) to 25(21-25) Physical Damage',
    'Adds (16-22) to (35-40) Physical Damage',
    '32% reduced Attribute Requirements',
]

def convertModTextToGeneric(mod_text):
    lines = mod_text.split('\n')
    generic_lines = []
    for l in lines:
        # https://regex101.com/r/AjXLZe/1
        quantity_matches = validateAndReturn(l, r'([\d\.]+)?(\([\d\-\.].*?\))?')
        quantity_matches = [q for q in quantity_matches if q != ('', '')]
        if len(quantity_matches) == 0:
            # Non number modification like "Hits have Culling Strike"
            generic_lines.append(l)
        else:
            # Some number modification like "Adds 20(20-26) to 47(40-47) Physical Damage"
            # For description, replace number data with X, so output is "Adds X to X Physical Damage"
            # https://regex101.com/r/05b4zw/1
            output_description = []
            last_idx = 0
            for match in quantity_matches:
                full_match = ''.join(match)
                start_index = l.index(full_match)
                end_index = start_index + len(full_match)
                if start_index > last_idx:
                    output_description.append(l[last_idx:start_index])
                output_description.append('X')
                last_idx = end_index
            if last_idx < len(l):
                output_description.append(l[last_idx:])
            output_description = ''.join(output_description)
    
            generic_lines.append(output_description)
    
    return '\n'.join(generic_lines)


for mod_text in mod_text_examples:
    g = convertModTextToGeneric(mod_text)
    print(g)
    print()

X% increased Physical Damage
+X to Accuracy Rating

Adds X to X Physical Damage

Adds X to X Physical Damage

X% reduced Attribute Requirements



In [179]:
with open(data_path / 'base_items.json') as f:
    base_items = json.load(f)
print(len(base_items))

4756


In [180]:
print(base_items['Metadata/Items/Weapons/OneHandWeapons/OneHandAxes/OneHandAxe18'])

{'domain': 'item', 'drop_level': 61, 'implicits': [], 'inventory_height': 3, 'inventory_width': 2, 'inherits_from': 'Metadata/Items/Weapons/OneHandWeapons/OneHandAxes/AbstractOneHandAxe', 'item_class': 'One Hand Axe', 'name': 'Reaver Axe', 'properties': {'armour': None, 'energy_shield': None, 'evasion': None, 'movement_speed': None, 'block': None, 'description': None, 'directions': None, 'stack_size': None, 'stack_size_currency_tab': None, 'full_stack_turns_into': None, 'charges_max': None, 'charges_per_use': None, 'duration': None, 'life_per_use': None, 'mana_per_use': None, 'attack_time': 833, 'critical_strike_chance': 500, 'physical_damage_max': 114, 'physical_damage_min': 38, 'range': 11, 'mana_burn_ms': None, 'cooldown_ms': None, 'monster_id': None, 'monster_ability_text': None, 'monster_category': None}, 'release_state': 'released', 'tags': ['axe', 'one_hand_weapon', 'onehand', 'weapon', 'default'], 'visual_identity': {'dds_file': 'Art/2DItems/Weapons/OneHandWeapons/OneHandAxes/O

In [205]:
reaver_class = base_items['Metadata/Items/Weapons/OneHandWeapons/OneHandAxes/OneHandAxe18']
reaver_class

{'domain': 'item',
 'drop_level': 61,
 'implicits': [],
 'inventory_height': 3,
 'inventory_width': 2,
 'inherits_from': 'Metadata/Items/Weapons/OneHandWeapons/OneHandAxes/AbstractOneHandAxe',
 'item_class': 'One Hand Axe',
 'name': 'Reaver Axe',
 'properties': {'armour': None,
  'energy_shield': None,
  'evasion': None,
  'movement_speed': None,
  'block': None,
  'description': None,
  'directions': None,
  'stack_size': None,
  'stack_size_currency_tab': None,
  'full_stack_turns_into': None,
  'charges_max': None,
  'charges_per_use': None,
  'duration': None,
  'life_per_use': None,
  'mana_per_use': None,
  'attack_time': 833,
  'critical_strike_chance': 500,
  'physical_damage_max': 114,
  'physical_damage_min': 38,
  'range': 11,
  'mana_burn_ms': None,
  'cooldown_ms': None,
  'monster_id': None,
  'monster_ability_text': None,
  'monster_category': None},
 'release_state': 'released',
 'tags': ['axe', 'one_hand_weapon', 'onehand', 'weapon', 'default'],
 'visual_identity': {'d

In [181]:
# Construct base_item_to_iclass mapping
print_counter = 0
max_print = 20
# print(base_items['Metadata/Items/Weapons/OneHandWeapons/OneHandAxes/OneHandAxe1'].keys())
base_item_to_iclass = defaultdict(list)
for k, v in base_items.items():
    base_item_to_iclass[v['name']].append(k)
    
    short_key = k.split('/')[-1]
    if print_counter < max_print:
        if 'one_hand_weapon' in v['tags'] and short_key.startswith('OneHand'):
            print(short_key, v['tags'])
            print_counter += 1

OneHandAxe1 ['axe', 'one_hand_weapon', 'onehand', 'weapon', 'default']
OneHandAxe2 ['axe', 'one_hand_weapon', 'onehand', 'weapon', 'default']
OneHandAxe3 ['axe', 'one_hand_weapon', 'onehand', 'weapon', 'default']
OneHandAxe4 ['axe', 'one_hand_weapon', 'onehand', 'weapon', 'default']
OneHandAxe5 ['axe', 'one_hand_weapon', 'onehand', 'weapon', 'default']
OneHandAxe6 ['axe', 'one_hand_weapon', 'onehand', 'weapon', 'default']
OneHandAxe7 ['axe', 'one_hand_weapon', 'onehand', 'weapon', 'default']
OneHandAxe8 ['axe', 'one_hand_weapon', 'onehand', 'weapon', 'default']
OneHandAxeM1 ['not_for_sale', 'maraketh', 'axe', 'one_hand_weapon', 'onehand', 'weapon', 'default']
OneHandAxe9 ['axe', 'one_hand_weapon', 'onehand', 'weapon', 'default']
OneHandAxe10 ['axe', 'one_hand_weapon', 'onehand', 'weapon', 'default']
OneHandAxe11 ['axe', 'one_hand_weapon', 'onehand', 'weapon', 'default']
OneHandAxe12 ['axe', 'one_hand_weapon', 'onehand', 'weapon', 'default']
OneHandAxe13 ['axe', 'one_hand_weapon', 'oneh

In [200]:
matching_ids_for_text = defaultdict(list)
for mod_id, info in j.items():
    if mod_id.startswith('WeaponTree'):
        # Ignore Crucible mods
        continue
    elif info.get('generation_type', 'N/A') in {'unique', 'scourge_benefit'}:
        # Ignore various impossible sources or things out of league
        continue
    else:
        # Ignore mods that cannot spawn
        pass # A lot of useful mods like essence and beastcraft do not have spawn weights
        # spawn_weights = info.get('spawn_weights')
        # if spawn_weights is not None:
        #     if not any([w['weight'] > 0 for w in spawn_weights]):
        #         continue
    
    # Convert to generic form
    mod_text = info['text']
    if mod_text is not None:
        g = convertModTextToGeneric(mod_text)
        matching_ids_for_text[g].append(mod_id)
    else:
        # There are lot of these but mostly seem to be uniques or monster mods, which aren't useful
        continue

def convertItemModsToId(item: PoEItem):
    output_mods = []
    for m in item.mods:
        text = m.stringDescription()
        possible_matches = matching_ids_for_text[text]

        if len(possible_matches) == 1:
            output_mods.append(possible_matches[0])
            continue
        
        # Check other data specified in original mod and see which ones match
        mined_info = {mod_id: j[mod_id] for mod_id in possible_matches}
    
        # Modifier title check (eg "of Craft")
        if m.title != '':
            possible_matches = [
                mod_id for mod_id in possible_matches
                if m.title == mined_info[mod_id]['name']
            ]

        if len(possible_matches) == 1:
            output_mods.append(possible_matches[0])
            continue
        
        # Check if weapon type can have said modifiers
        # If there are non-matching mods for "spawn_weighting" list then it is impossible to have the mod
        # TODO: ^^ Unfortunately this simple rule isn't true, it's probably some class OOP stuff
        # Look up iclass of item using base name
        iclass = base_item_to_iclass[item.base]
        if len(iclass) == 1:
            iclass = iclass[0]
            allowed_class_tags = set(base_items[iclass]['tags'])
            new_possible_matches = []
            for mod_id in possible_matches:
                mod_allowed_spawns = set([w['tag'] for w in mined_info[mod_id]['spawn_weights']])
                overlapping_tags = mod_allowed_spawns & allowed_class_tags
                print(overlapping_tags)
                if len(overlapping_tags) > 1: # excluding "default" tag
                    new_possible_matches.append(mod_id)
            possible_matches = new_possible_matches
        
        if len(possible_matches) == 1:
            output_mods.append(possible_matches[0])
            continue
        
        output_mods.append(possible_matches)
    return output_mods

In [199]:
chosen_item = recombs['00139.json']['input1']
for m in chosen_item.mods:
    print(f'"{m.stringDescription()}"')
print('==========')
mod_ids = convertItemModsToId(chosen_item)
# print(mod_ids)
print_keys = [
    'generation_type',
    'text',
]
print(mod_ids)
for mod_matches in mod_ids:
    if isinstance(mod_matches, str):
        cprint(mod_matches, 'green')
        print('  -----')
    else:
        for mod_id in mod_matches:
            print('  ', mod_id)
            print('     ', end='')
            for k in print_keys:
                print(j[mod_id].get(k), end=' | ')
            print()
        if len(mod_matches) == 0:
            cprint('NO MATCH', 'red')
        print('  -----')

"Adds X to X Fire Damage"
"Adds X to X Physical Damage"
"Socketed Gems are Supported by Level X Chance To Bleed — Unscalable Value
X% chance to cause Bleeding on Hit"
"+X% to Chaos Resistance"
"X% reduced Attribute Requirements"
{'default', 'axe'}
{'default', 'one_hand_weapon', 'axe'}
{'default'}
{'default'}
{'default'}
{'default'}
{'default', 'one_hand_weapon'}
{'default'}
[['LocalAddedFireDamage4', 'LocalAddedFireDamageTwoHand4'], 'LocalAddedPhysicalDamage5', [], 'ChaosResist3', 'ReducedLocalAttributeRequirements2']
   LocalAddedFireDamage4
     prefix | Adds (17-24) to (35-41) Fire Damage | 
   LocalAddedFireDamageTwoHand4
     prefix | Adds (32-44) to (65-76) Fire Damage | 
  -----
LocalAddedPhysicalDamage5
  -----
NO MATCH
  -----
ChaosResist3
  -----
ReducedLocalAttributeRequirements2
  -----


In [105]:
text_to_id = {}
print_counter = 0
max_print = 3
for mod_id, info in j.items():
    if info['text'] is not None:
        lt = info['text'].lower()
        if mod_id.startswith('LocalAddedPhysicalDamage') and info.get('generation_type', '') != 'unique':
            print(mod_id)
            print(info)
            print()
            print_counter += 1
            if max_print == print_counter:
                break

LocalAddedPhysicalDamage1
{'adds_tags': ['has_attack_mod'], 'domain': 'item', 'generation_type': 'prefix', 'generation_weights': [{'tag': 'has_attack_mod', 'weight': 100}, {'tag': 'has_caster_mod', 'weight': 65}, {'tag': 'default', 'weight': 100}], 'grants_effects': [], 'groups': ['PhysicalDamage'], 'implicit_tags': ['physical_damage', 'damage', 'physical', 'attack'], 'is_essence_only': False, 'name': 'Glinting', 'required_level': 2, 'spawn_weights': [{'tag': 'one_hand_weapon', 'weight': 1000}, {'tag': 'default', 'weight': 0}], 'stats': [{'id': 'local_minimum_added_physical_damage', 'max': 1, 'min': 1}, {'id': 'local_maximum_added_physical_damage', 'max': 3, 'min': 2}], 'text': 'Adds 1 to (2-3) Physical Damage', 'type': 'LocalPhysicalDamage'}

LocalAddedPhysicalDamage2
{'adds_tags': ['has_attack_mod'], 'domain': 'item', 'generation_type': 'prefix', 'generation_weights': [{'tag': 'has_attack_mod', 'weight': 100}, {'tag': 'has_caster_mod', 'weight': 65}, {'tag': 'default', 'weight': 100}

In [106]:
j['Strength1']

{'adds_tags': [],
 'domain': 'item',
 'generation_type': 'suffix',
 'generation_weights': [],
 'grants_effects': [],
 'groups': ['Strength'],
 'implicit_tags': ['attribute'],
 'is_essence_only': False,
 'name': 'of the Brute',
 'required_level': 1,
 'spawn_weights': [{'tag': 'ring', 'weight': 1000},
  {'tag': 'amulet', 'weight': 1000},
  {'tag': 'belt', 'weight': 1000},
  {'tag': 'str_armour', 'weight': 1000},
  {'tag': 'str_dex_armour', 'weight': 1000},
  {'tag': 'str_int_armour', 'weight': 1000},
  {'tag': 'str_dex_int_armour', 'weight': 1000},
  {'tag': 'sword', 'weight': 1000},
  {'tag': 'mace', 'weight': 1000},
  {'tag': 'sceptre', 'weight': 1000},
  {'tag': 'staff', 'weight': 1000},
  {'tag': 'axe', 'weight': 1000},
  {'tag': 'default', 'weight': 0}],
 'stats': [{'id': 'additional_strength', 'max': 12, 'min': 8}],
 'text': '+(8-12) to Strength',
 'type': 'Strength'}

In [178]:
for k, v in base_item_to_iclass.items():
    if len(v) > 1:
        print(k, v)

Corrupt ['Metadata/Items/Currency/CurrencyIncursionCorrupt1', 'Metadata/Items/Currency/CurrencyIncursionCorrupt2', 'Metadata/Items/Currency/CurrencyIncursionCorruptGem']
Silver Coin ['Metadata/Items/Currency/CurrencySilverCoin', 'Metadata/Items/Currency/CurrencyAncestralSilverCoin']
Fine Delirium Orb ['Metadata/Items/Currency/CurrencyAfflictionOrbCurrency', 'Metadata/Items/Currency/CurrencyAfflictionOrbProphecies']
Delirium Orb ['Metadata/Items/Currency/CurrencyAfflictionOrbGeneric', 'Metadata/Items/Currency/CurrencyAfflictionOrbHardMode']
Primitive Alchemical Resonator ['Metadata/Items/Delve/DelveSocketableCurrencyUpgrade1', 'Metadata/Items/Delve/DelveStackableSocketableCurrencyUpgrade1']
Potent Alchemical Resonator ['Metadata/Items/Delve/DelveSocketableCurrencyUpgrade2', 'Metadata/Items/Delve/DelveStackableSocketableCurrencyUpgrade2']
Powerful Alchemical Resonator ['Metadata/Items/Delve/DelveSocketableCurrencyUpgrade3', 'Metadata/Items/Delve/DelveStackableSocketableCurrencyUpgrade3']